In [2]:
import numpy as np
import pandas as pd
import time
from datetime import datetime
from glob import glob
import os
from math import ceil

import ccxt

from tp_utils.data_provider import get_symbols, market_to_symbol, load_data_from_exchange
from tp_config import *

pd.set_option('display.precision', 9)

In [4]:
pair = 'USDT'
timeframes = {'1m':1, '3m':3, '5m':5, '15m':15}
#timeframes = {'30m':30, '1h':60} #'2h':120, '4h':240, '6h':480, '12h':960, '1d':1920}
#timeframes = {'1h':60}# '2h':120, '4h':240, '6h':480, '12h':960, '1d':1920}
timeframes = {'4h':240, '6h':480, '12h':960, '1d':1920}
timeframes = {'4h':240}
exchange = ccxt.bybit()
symbol_list = get_symbols(exchange, pair)

symbols = [x for x in symbol_list if ":" not in x]

In [3]:
index_assets = pd.read_csv('Portfolio/index_assets.csv')['asset'].tolist()
index_low_assets = pd.read_csv('Portfolio/index_low_assets.csv')['asset'].tolist()
full_assets = index_assets + index_low_assets

markets = [x.replace('/','-') for x in symbol_list if x.split('/')[0] != 'BTC']

main = ['USDT']

assets = full_assets
#markets = main
markets = [s + '-' + pair for s in assets] 

In [8]:
markets = [x.replace('/','-') for x in symbols]
data_path = DATA_PATH_CRYPTO + '/' + pair

now = exchange.milliseconds()
to_timestamp = now
start_timestamp = exchange.parse8601('2024-01-01 00:00:00')

### 0. Data from exchange

In [9]:
pair = "USDC"
data_path = DATA_PATH_CRYPTO + '/' + pair
markets = ["SOL-USDC"]
timeframes = {'5m':5, '15m':15}

In [10]:
load_data_from_exchange(exchange, markets, timeframes, start_timestamp,  data_path, 10)

1704067200000 2024-01-01 04:00:00
SOL-USDC
5m
2024-04-07 15:40:00
15m
2024-04-07 15:30:00
Done
Time taken = 0.004  hours


### PyBit

In [47]:
import pybit

In [ ]:
from pybit.unified_trading import HTTP
session = HTTP()
print(session.get_kline(
    category="spot",
    symbol="SOLUSDC",
    interval=5,
    start=1706716267000,
))

In [48]:
session = HTTP(testnet=True)
print(session.get_instruments_info(
    category="linear",
    symbol="JUPUSDT",
))

{'retCode': 0, 'retMsg': 'OK', 'result': {'category': 'linear', 'list': [{'symbol': 'JUPUSDT', 'contractType': 'LinearPerpetual', 'status': 'Trading', 'baseCoin': 'JUP', 'quoteCoin': 'USDT', 'launchTime': '1706716267000', 'deliveryTime': '0', 'deliveryFeeRate': '', 'priceScale': '4', 'leverageFilter': {'minLeverage': '1', 'maxLeverage': '50.00', 'leverageStep': '0.01'}, 'priceFilter': {'minPrice': '0.0001', 'maxPrice': '199.9998', 'tickSize': '0.0001'}, 'lotSizeFilter': {'maxOrderQty': '30000', 'minOrderQty': '1', 'qtyStep': '1', 'postOnlyMaxOrderQty': '30000', 'maxMktOrderQty': '15000'}, 'unifiedMarginTrade': True, 'fundingInterval': 240, 'settleCoin': 'USDT', 'copyTrading': 'none', 'upperFundingRate': '0.006', 'lowerFundingRate': '-0.006'}], 'nextPageCursor': ''}, 'retExtInfo': {}, 'time': 1710779933743}
